# Equity Market Data as they appear on organised markets
## (and what to do with it)

Listed companies have their stock prices quoted on one or several organised markets. There are a lot of markets (e.g. https://en.wikipedia.org/wiki/List_of_stock_exchanges_in_the_Americas) and a standard issue for financial institutions wanting to trade efficiently is to get accesss to as many as necessary.

In addition, for the companies companies that are part of a reference index (S&P500, EUROSTOXX50, NIKKEI,etc.), there are also listed markets for standard options. unlike stock prices, these listed markets are not freely visible, except for the options listed on Nasdaq, which we will look at today.

For financial institution (banks mostly), there are other instruments available, which are traded directly between participants, through brokers. There are obviouly not visible.

## Short presentation of Nasdaq data

The data freely available are published with a delay of !5 minutes (classical approach, unlike FX spot which is available as a live quote). For example:
https://www.nasdaq.com/market-activity/stocks/amzn/option-chain
To retrieve these data, there are several python scrapping tools available. the most common ones use the data replicated on yahoo finance, which also provides many other information on the companies. In what follows, I used yFinance (https://pypi.org/project/yfinance/).

In [ ]:
import yfinance as yf

ticker = 'DIS'

dict=yf.Ticker(ticker).info
for key in dict:
    print(key,":",dict[key])

In [ ]:
import pandas as pd

ticker = 'AMZN'
data_date = "2022-10-31"

#data loading (retrieved at market close)
raw_data_folder = "DATA\\raw_data_" + data_date + "\\"
filename = raw_data_folder + ticker + " - yFinance raw data - " + data_date +" @ close.json"
options_yf_pddf = pd.read_json(filename)
options_yf_pddf["lastTradeDate"] = pd.to_datetime(options_yf_pddf["lastTradeDate"], unit='ms') #required format adjustment

print("Number of different contracts traded on",data_date, ":",len(options_yf_pddf))
options_yf_pddf.head()

#### Closer look at the data

In [ ]:
expiry_date_list=options_yf_pddf["expiryDate"].unique()
print(expiry_date_list)

In [ ]:
for expiry_date in expiry_date_list:
    options_at_given_expiry_yf_pddf = options_yf_pddf.loc[options_yf_pddf["expiryDate"]==expiry_date]
    print("Number of different contracts traded on",data_date, "for contracts expiring on", expiry_date, ":", len(options_at_given_expiry_yf_pddf))

Market participants have access to much more contracts, for bespoke strikes and maturity, and also to other type of instruments, like variance swaps.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

expiry_date = "2023-06-16"

options_at_given_expiry_yf_pddf = options_yf_pddf.loc[options_yf_pddf["expiryDate"]==expiry_date]
price_at_close = options_at_given_expiry_yf_pddf["last close"].mean()
calls_at_given_expiry_yf_pddf = options_yf_pddf.loc[(options_yf_pddf["expiryDate"]==expiry_date) & (options_yf_pddf["isCall"]==True)]
puts_at_given_expiry_yf_pddf = options_yf_pddf.loc[(options_yf_pddf["expiryDate"]==expiry_date) & (options_yf_pddf["isCall"]==False)]

plt.figure(figsize = (8, 8))
plt.plot(np.array(calls_at_given_expiry_yf_pddf['strike'])/price_at_close, calls_at_given_expiry_yf_pddf['impliedVolatility'], 'b', label="Implied vol for calls", linewidth=1)
plt.plot(np.array(puts_at_given_expiry_yf_pddf['strike'])/price_at_close, puts_at_given_expiry_yf_pddf['impliedVolatility'], 'r', label="Implied vol for puts", linewidth=1)
plt.title("yFinance implied volatilities")
plt.xlabel("Strike in spot moneyness")
plt.legend()
plt.grid()
plt.show()

The problem of "separated" implied volatilities curves computed using put prices and call prices comes from the use of an incorrect forward when solving the inverse problem. this information is useless (it's a computation form yahoo finance, which is NOT a market information).

It's not a problem. It just mean that we have to compute the implied volatility directly from the real observable market data: the option prices. 

But first, let's agree on what we call implied volatilit....

## Which implied volatility is observable in the market? Black-Scholes vs. Black Formula

The forward of an equity at time $T$ as seen from $t<T$ is given by:
$$
F(t,T)=S(t)\frac{RF(t,T)}{DF(t,T)}-\sum_{t<{t_{ex_i}}\leq{T}}^{T}{d_i}\frac{RF(t_{p_i},T)}{DF(t_{p_i},T)}.
$$

where:

- $d_i$ is a dividend detached at $t_{ex_i}$ (meaning the owner of a share will receive the dividend if he/she owns it at $t_{ex_i}-1$ day, the "$ex_i$" stands for "exclusion date for dividend $i$"), and paid at $t_{p_i}\geq{t_{ex_i}}$. We do not make any hypothesis on the value of the dividend here, wether it is deterministic, or proportional to the expected value of the share price at announcement date (official communication prior to the exclusion date). 

 - $DF(t,T)$ is the discount factor from $t$ to $T$. If $r_s$ is the instantaneous spot rate, $DF(t,T)=E_t^Q\left[\exp({-\int_{t}^{T}r_s ds})\right]$, $Q$ being the risk neutral measure.

 - $RF(t,T)$ is the repo factor from $t$ to $T$. It correspond to the borrowing cost of the share. We can represent it as $RF(t,T)=\exp({-\int_{t}^{T}b_sds})$ (it would be painless to make it stochastic as well, but apart from equity financing desks, this is generally useless).

With this definition of the forward, one can see that the "real" word is more complicated than the Black-Scholes formulation, where the drift of the underlying asset under the risk neutral measure is deterministic, equal to $r$. Fortunatly, through a proper change of measure, and an additional hypothesis we can rewrite it in a much more generic yet simple form, commonly known by practitionners as Black model.

First, we need to rewrite the forward in a more usable way (no intermediate cash flows between $t$ and $T$):
$$
F(t,T)=S(t)\frac{RF(t,T).DivF(t,T)}{DF(t,T)}.
$$

where:

$DivF(t,T)$ is expresssed as $DivF(t,T)=\exp({-\int_{t}^{T}q_sds})$, with $q$ the dividend yield, constructed to match the forward given by the general expression at specific dates (the expiry dates of the vanilla option contracts typically).

### Black formula
Suppose we are at $t_0$. We define $Q^T$, the forward measure at date $T$ by:
$$
\frac{dQ^T}{dQ}\bigg\rvert_{t}=\frac{B_{t_0}(t)DF(t,T)}{DF(t_0,T)}
$$
where $B_{t_0}(t)=\exp({-\int_{t_0}^{t}r_s ds})$.

The price at $t_0$ of an European option of strike $K$, expiring and paid at $T$, on an observable asset $S(T)$ is:
$$
\boxed{
\begin{align*}
\text{EuropeanOptionPrice}_{t_0}(K,T) &=E^Q[B_{t_0}(T)\cdot \max(\epsilon \cdot (S(T)-K); 0)] \\
                                    &=DF(t_0,T)\cdot E^{Q^T}[\max(\epsilon \cdot (S(T)-K); 0)]
\end{align*}
}
$$
where $\epsilon$ is equal to 1 for a call option and -1 for a put option.

If $F(t_0,T)$ is given by the expression above, we define the dynamic, under $Q^T$ of the forward $F(t,T)$, $t_0 \leq t \leq T$, by:
$$
\frac{dF(t,T)}{F(t,T)}=\sigma_{t}^{F,T}dW_{t}^{F,T}.
$$
Since $S_T=F(T,T)$, the undiscounted price at $t_0$ of an European option of strike $K$, expiring at $T$ in that model is given by:
$$
\boxed{
\begin{align*}
\text{BlackFormula}(t_0,F(t_0,T),T,K,\Sigma^F(K,T),\epsilon) &= E^{Q^T}[\max(\epsilon \cdot (F(T,T)-K); 0)] \\
                                                           &= \epsilon \cdot \left( F(t_0,T)\cdot\mathcal{N}(\epsilon \cdot d_1) - K\cdot\mathcal{N}(\epsilon \cdot d_2)\right), \\
d_1 &= \frac{-k}{\Sigma^F(K,T)\sqrt{T-t_0}} + \frac{1}{2}\Sigma^F(K,T)\sqrt{T-t_0}, \\
d_2 &= d_1 - \Sigma^F(K,T)\sqrt{T-t_0}, \\
k &= \log\left(\frac{F(t_0,T)}{K}\right).
\end{align*}
}
$$
where:
 - $\epsilon$ is equal to 1 for a call option and -1 for a put option,
 - $\mathcal{N}()$ the standard normal cumulative function,
 - $\Sigma^F(K,T)$ the implied volatility for the option of strike $K$ expiring at $T$ (meaning that, if we define the model corresponding to the pricing of a European option of strike $K$ by $dF(t,T)/F(t,T)=\sigma_{t}^{F,K,T}dW_{t}^{F,T}$, $\Sigma^F(K,T)^2 \cdot (T-t_0) = \int_{t_0}^{T}{\left(\sigma_{u}^{F,K,T}\right)^2du}$ ).
 
The market price at $t_0$ of this European option, settled at $T_p\geq T$, it given by:
$$
\boxed{
\text{PV}(t_0,F(t_0,T),T,K,\Sigma(K,T),\epsilon, DF(t_0,T_p)) = DF(t_0,T_p) \cdot \text{BlackFormula}(t_0,F(t_0,T),T,K,\Sigma^F(K,T),\epsilon)
}
$$

**The implied volatility we observe on the market and compute by solving the inverse problem corresponds to the Black volatility.**

Note that there is a small abuse here: if $T_p\neq T$, in the presence of stochastic rates, the correct measure to use in this model is $Q^{T_p}$, under wich there is a small convexity adjustment for $F(t,T)$ (a drift term whose formulation depends on the model used for rates). It is common practice to neglect it when the payment is a few days after the expiry (standard settlement rule).

### Exercise

Suppose that rates are stochastic, and that $\frac{dB(t,T)}{B(t,T)}=...dt+\Gamma (t,T)dW_{t}^{B}$, where $B(t,T)=\exp({-\int_{t}^{T}r_s ds})$.

Suppose also that $\frac{dS_t}{S_t}=(r_t-b_t-q_t)dt + \sigma_{t}^{S}dW_{t}^{S}$, where the repo rate and dividend yield are deterministic.

Suppose finally that $d<W^S,W^B>_t=\rho dt$.

1. From the equations above, express $\Sigma^F(T)$, the implied volatility at $T$ observed from option market prices as a function of $\sigma_{t}^{S}$, $\Gamma (t,T)$ and $\rho$ (no smile, or rather we consider the spot and forward processes associated with a strike $K$ for expiry $T$).
2. Calibration of a simple equity/rate model: suppose we observe in the market the discount factors, the volatility of the rates (e.g. calibrated from caps/floors or swaptions) from which we have deduced an expression for the dynamic of the zero coupon bond as above, the forward of an asset, the Black implied volatility for several maturities $T_i$ (no smile), and the correlation between rates and equities (if the short rate was described by a vasicek model, and the asset by a lognormal model, how would you calibrate this correlation on historical data?). Propose a calibration algorithm for $\sigma^S_t$.
3. By analogy, for a FX Spot process with both domestic and foreign rates defined by stochastic processes as above, what would be the formula linking the market implied volatility to the rate volatilities, the FX Spot idiosyncratic volatility, and the correlations between the three risk factors?

### Closing comment
The Black-Scholes framework is unrealistic but, with a proper change of measure, we have shown that we can use an almost identical formula to compute European option prices and define an implied volatility from it, which aggregates the uncertainty coming from the rates with the idiosyncratic movement of the stock price.

In term of dynamic hedging, what the Black model implies is that it is possible to perfectly replicate a European option in a context where both rates and assets are stochastics (under the hypothesis of a fixed volatility for the forward) by executing the delta hedge  with the forward at the maturity of the European option, and by funding the self financing stategy with a term deposit to the settlement date. In practice, this is impossible, which means that equity traders are exposed to the fluctuation of the interest rates. That is why they must hedge their sensitivity to the interest rates ($rho$).

Moreover, this perfect replication strategy can only be used for derivative claims referencing a single observation date. If multiple observation dates are required, one needs to properly specify the stock process and the rate process to account for convexity adjustments. Obviously, the simplest approach is to use deterministic rates, which makes all forward measures identical to the risk neutral measure (this is what we do when we compute local volatilities using Dupire formula for example). 

## Ingredients for the computation of the Black implied volatility

From the formula above, we need three additional elements, apart from the option price, to solve the inverse problem:
 - the time to maturity $T-t_0$, as a year fraction, which we can compute as long as we are clear on which daycount convention to use (https://en.wikipedia.org/wiki/Day_count_convention). **It is common practice to express all volatility time in act_365**, meaning to count the number of days between $t_0$ and $T$ and divide it by 365.
 - the discount factor to maturity. In the case of US stocks, the market curve for discounting options is the SOFR curve, for which there is an overnight rate published by the Federal Reserve of New York (https://www.newyorkfed.org/markets/reference-rates/sofr), quotes available for Term instruments (https://www.cmegroup.com/market-data/cme-group-benchmark-administration/term-sofr.html), and swap quotes available through broker markets. As a general rule, never try to imply it from option quotes.
 - the forward to maturity $F(t_0, T)$. There are two possibilities here:
    - to use the formula above if we have a view on expected repo rate and dividends (some information is available on yahoo, but be very careful: it is called "dividendRate" and it is in fact a cash amount, and it is not a market implied information),
    - to use call/put parity when we have both contracts traded for the same strike (recall that $\text{call}(K,T)-\text{put}(K,T)=DF(t_0,T)(F(t_0,T)-K)$ is model independent).

Here is my personnal cooking recipe for the computation of the forward:
1. take a dividend yield hypothesis, set repo rate to 0, and use the market standard for interest rates (SOFR here) to estimate the value of the forward,
2. look for options with strikes around that estimated forward where both calls and puts are traded for the same strike,
3. compute the forward by parity from these options.

## Data Selection
The final question we need to answer before calculating the implied volatility is which contracts do we keep, which ones do we exclude.

In [ ]:
expiry_date = "2022-11-18"

options_at_given_expiry_yf_pddf = options_yf_pddf.loc[options_yf_pddf["expiryDate"]==expiry_date].sort_values(by=['strike'], ascending = [False])
pd.set_option('display.max_rows', len(options_at_given_expiry_yf_pddf))
options_at_given_expiry_yf_pddf.head(len(options_at_given_expiry_yf_pddf))

First of all, we only keep the information from out of the money options.

Then we must remove all options which have a bid at 0.

Finally, a common practice is also to filter out all the options that have an absolute delta below a treshold (I usually take 0.1%, keeping 99.8% of the distribution), and all those where the bid-ask spread in volatility is too wide (for example, from the distribution of bid-ask spreads, we can decide to keep all the options whose bid-ask spread is within the 95% percentile).

## Results

In [ ]:

from Equity_data_generation_from_yFinance import yFinance_generate_refined_implied_volatility_surface_dictionnary, yFinance_raw_to_refined_data_sanity_check,  yFinance_Equity_IV_graph


SOFR_daycountfractions = [0.025, 0.044444444444444446, 0.06388888888888888, 0.08888888888888889, 0.17777777777777778, 0.2611111111111111, 0.3388888888888889, 0.42777777777777776, 0.5083333333333333, 0.5944444444444444, 0.6805555555555556, 0.7638888888888888, 0.8583333333333333, 0.9333333333333333, 1.0194444444444444, 1.275, 1.525, 1.7805555555555554, 2.0416666666666665, 3.0527777777777776, 4.063888888888889, 5.0777777777777775, 6.094444444444444, 7.108333333333333, 8.127777777777778, 9.13888888888889, 10.152777777777779, 11.166666666666666, 12.180555555555555, 15.225, 20.3, 25.375, 30.45, 40.58888888888889, 50.736111111111114]
SOFR_dfs = [0.99908377, 0.99834724, 0.99761118, 0.99666563, 0.99299067, 0.9893521, 0.98568173, 0.98146704, 0.97759009, 0.9734431, 0.96932935, 0.96538615, 0.96099668, 0.95754721, 0.95364084, 0.94251165, 0.93228499, 0.92250253, 0.91307201, 0.87999494, 0.84928716, 0.81983717, 0.79122344, 0.76392464, 0.73751515, 0.71183102, 0.68629207, 0.66149736, 0.63743818, 0.5707853, 0.4832476, 0.4235979, 0.3760151, 0.31293885, 0.27311336]

expiry_date = "2023-06-16"

equity_data = yFinance_generate_refined_implied_volatility_surface_dictionnary(options_yf_pddf, SOFR_daycountfractions, SOFR_dfs, expiry_date_list=[expiry_date], forced_dividend_yield=None, log=False)
print("forward:", equity_data["implied_volatility_surface"][0]["reference_forward"])

calls_at_given_expiry_yf_pddf = options_yf_pddf.loc[(options_yf_pddf["expiryDate"]==expiry_date) & (options_yf_pddf["isCall"]==True)]
puts_at_given_expiry_yf_pddf = options_yf_pddf.loc[(options_yf_pddf["expiryDate"]==expiry_date) & (options_yf_pddf["isCall"]==False)]
price_at_close = calls_at_given_expiry_yf_pddf["last close"].mean()

plt.figure(figsize = (10, 10))
plt.plot(np.array(calls_at_given_expiry_yf_pddf['strike'])/price_at_close, calls_at_given_expiry_yf_pddf['impliedVolatility'], 'lightgrey', label="yF Implied vol for calls", linewidth=1)
plt.plot(np.array(puts_at_given_expiry_yf_pddf['strike'])/price_at_close, puts_at_given_expiry_yf_pddf['impliedVolatility'], 'darkgrey', label="yF Implied vol for puts", linewidth=1)
plt.plot(np.array(equity_data["implied_volatility_surface"][0]["strikes"])/price_at_close, equity_data["implied_volatility_surface"][0]["mid_implied_volatilities"], 'darkblue', label="Mid implied vol computed", linewidth=1)
plt.plot(np.array(equity_data["implied_volatility_surface"][0]["strikes"])/price_at_close, equity_data["implied_volatility_surface"][0]["bid_implied_volatilities"], 'red', label="Bid implied vol computed", linewidth=1)
plt.plot(np.array(equity_data["implied_volatility_surface"][0]["strikes"])/price_at_close, equity_data["implied_volatility_surface"][0]["ask_implied_volatilities"], 'red', label="Ask implied vol computed", linewidth=1)
plt.title("yFinance implied volatilities")
plt.xlabel("Strike in spot moneyness")
plt.legend()
plt.grid()
plt.show()

In [ ]:
equity_data = yFinance_generate_refined_implied_volatility_surface_dictionnary(options_yf_pddf, SOFR_daycountfractions, SOFR_dfs, expiry_date_list=None, forced_dividend_yield=None, log=False)
yFinance_Equity_IV_graph.lines_3D(ticker, equity_data)

more examples...

## Final words

What you need to remember from this presentation:
- Which data is available, which data is reserved to market participants via brokers.
- How to compute implied volatility from market prices. 

excursus: Black formula and Black implied volatility in Python.

Going further: calibration exercise with the SVI parametric form.